#<font color="Red">MDX Colab with new UVR models</font><br>

It downloads all models automatically. <br>No manual models.py replacing necessary for various models

<sup><br>If you have "vocals.onnx not found" check ForceUpdate or delete MDX_Colab folder from your GDrive, terminate the session and start over.

<sup><br>Models provided are from [Kuielab](https://github.com/kuielab/mdx-net-submission/), [UVR](https://github.com/Anjok07/ultimatevocalremovergui/) and [Kim](https://github.com/KimberleyJensen/) <br> (you can support UVR [here](https://www.buymeacoffee.com/uvr5/vip-model-download-instructions) and [here](https://boosty.to/uvr)).</sup></br>
<sup><br>Notebook by [Audio Hacker](https://www.youtube.com/channel/UC0NiSV1jLMH-9E09wiDVFYw/), modified by Audio Separation community.</sup></br> 



In [ ]:
initialised = True
from time import sleep
from google.colab import output
from google.colab import drive

import sys
import os
import shutil
import psutil
import glob

ai = 'https://github.com/kae0-0/Colab-for-MDX_B'
ai_version = 'https://github.com/kae0-0/Colab-for-MDX_B/raw/main/v'
onnx_list = 'https://raw.githubusercontent.com/kae0-0/Colab-for-MDX_B/main/onnx_list'
#@title Initialise MDX
#@markdown Mounting to drive
mount_to_drive = True #@param {type:"boolean"}
#@markdown Mounting path (used only if mount_to_drive is ticked)
mount_path = '/content/drive/MyDrive' #@param ["snippets:","/content/drive/MyDrive","/content/drive/Shareddrives/<your shared drive name>", "/content/drive/Shareddrives/Shared Drive"]{allow-input: true}
ForceUpdate = False #@param {type:"boolean"}
class h:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')
    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout
def get_size(bytes, suffix='B'): # read ram
    global svmem
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f'{bytes:.2f}{unit}{suffix}'
        bytes /= factor
    svmem = psutil.virtual_memory()
def console(t):
    get_ipython().system(t)
def LinUzip(file): # unzip call linux, force replace
    console(f'unzip -o {file}')
#-------------------------------------------------------
def getONNX():
    console(f'wget {onnx_list} -O onnx_list')
    _onnx = open("onnx_list", "r")
    _onnx = _onnx.readlines()
    os.remove('onnx_list')
    for model in _onnx:
        _model = sanitize_filename(os.path.basename(model))
        console(f'wget {model}')
        LinUzip(_model)
        os.remove(_model)
    

def getDemucs(_path):
    #https://dl.fbaipublicfiles.com/demucs/v3.0/demucs_extra-3646af93.th
    root = "https://dl.fbaipublicfiles.com/demucs/v3.0/"
    model = {
        'demucs_extra': '3646af93'
    }
    for models in zip(model.keys(),model.values()):
        console(f'wget {root+models[0]+"-"+models[1]}.th -O {models[0]}.th')
    for _ in glob.glob('*.th'):
        if os.path.isfile(os.path.join(os.getcwd(),_path,_)):
            os.remove(os.path.join(os.getcwd(),_path,_))
        shutil.move(_,_path)
def mount(gdrive=False):
    if gdrive:
        try:
            drive.mount("/content/drive", force_remount=True)
        except:
            drive._mount("/content/drive", force_remount=True)
    else:
        pass
def toPath(path='local'):
    if path == 'local':
        os.chdir('/content')
    elif path == 'gdrive':
        os.chdir(mount_path)

def update():
    with h():
        console(f'wget {ai_version} -O nver')
    f = open('nver', 'r')
    nver = f.read()
    f = open('v', 'r')
    cver = f.read()
    if nver != cver or ForceUpdate:
        print('New update found! {}'.format(nver))
        os.chdir('../')
        print('Updating ai...',end=' ')
        with h():
            console(f'git clone {ai} temp_MDX_Colab')
            console('cp -a temp_MDX_Colab/* MDX_Colab/')
            console('rm -rf temp_MDX_Colab')
        print('done')
        os.chdir('MDX_Colab')
        print('Refreshing models...', end=' ')
        with h():
            getDemucs('model/')
            getONNX()
        print('done')
        output.clear()
        os.remove('v')
        os.rename("nver",'v')
        #os.chdir(f'{os.path.join(mount_path,"MDX_Colab")}')
    else:
        os.remove('nver')
        print('Using latest version.')

def past_installation():
    return os.path.exists('MDX_Colab')

def LoadMDX():
    console(f'git clone {ai} MDX_Colab')

#-------------------------------------------------------
# install requirements
print('Installing dependencies will take 45 seconds...',end=' ')

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('command not found') >= 0:
    svmem = psutil.virtual_memory()
    gpu_runtime = False
    with h():
        console('pip3 -q install onnxruntime==1.14.1')
else:
    gpu_runtime = True
    with h():
        console('pip3 -q install onnxruntime-gpu==1.14.1')
with h():
    deps = [
            'pathvalidate',
            'youtube-dl',
            'django'
    ]
    for dep in deps:
        console('pip3 -q install {}'.format(dep))
# import modules
#console('pip3 -q install torch==1.13.1')
console('pip3 -q install soundfile==0.12.1')
console('pip3 -q install librosa==0.9.1')
from pathvalidate import sanitize_filename
print('done')
if not gpu_runtime:
    print(f'GPU runtime is disabled. You have {get_size(svmem.total)} RAM.\nProcessing will be incredibly slow. 😈')
elif gpu_info.find('Tesla T4') >= 0:
    print('You got a Tesla T4 GPU. (speeds are around  10-25 it/s)')
elif gpu_info.find('Tesla P4') >= 0:
    print('You got a Tesla P4 GPU. (speeds are around  8-22 it/s)')
elif gpu_info.find('Tesla K80') >= 0:
    print('You got a Tesla K80 GPU. (This is the common gpu, speeds are around 2-10 it/s)')
elif gpu_info.find('Tesla P100') >= 0:
    print('You got a Tesla P100 GPU. (This is the Second to the fastest gpu, speeds are around  15-42 it/s)')
else:
    if gpu_runtime:
        print('You got an unknown GPU. Please report the GPU you got!')
        ! nvidia-smi
#console('pip3 -q install demucs')
#-------------------------------------------------------
# Scripting
mount(mount_to_drive)
toPath('gdrive' if mount_to_drive else 'local')
#check for MDX existence
if not past_installation():
    print('First time installation will take around 3-6 minutes.\nThis requires around 2-3 GB Free Gdrive space.\nPlease try not to interup installation process!!')
    print('Downloading AI...',end=' ')
    with h():
        LoadMDX()
    os.chdir('MDX_Colab')
    print('done')
    
    print('Downloading models...',end=' ')
    with h():
        getDemucs('model/')
        getONNX()
    if os.path.isfile('onnx_list'):
        os.remove('onnx_list')
    print('done')

else:
    os.chdir('MDX_Colab')
    update()

################
#outro
print('Success!')

#<font color="Red">Inference</font>

In [ ]:
#@markdown ### Print a list of tracks
for i in glob.glob('tracks/*'):
    print(os.path.basename(i))

In [ ]:
if not 'initialised' in globals():
    raise NameError('Please run the first cell first!! #scrollTo=H_cTbwhVq4K6')

#import all models metadata
import json
with open('model_data.json', 'r') as f:
  model_data = json.load(f)

# Modifiable variables
tracks_path = 'tracks/'
separated_path = 'separated/'

#@markdown ### Input track
#@markdown Enter any link/Filename (Upload your songs in tracks folder)
track = "Butterfly.wav" #@param {type:"string"}

#@markdown ---
#@markdown ### Models
ONNX = "MDX-UVR Ins Model Full Band 498 (HQ_2)" #@param ["off", "Karokee", "Karokee_AGGR", "Karokee 2", "baseline", "MDX-UVR Ins Model 415", "MDX-UVR Ins Model 418", "MDX-UVR Ins Model 464", "MDX-UVR Ins Model 496 - inst main-MDX 2.1", "Kim ft other instrumental model", "MDX-UVR Vocal Model 406", "MDX-UVR Vocal Model 427", "MDX-UVR-Kim Vocal Model (old)", "MDX-UVR-Kim Vocal Model 2 (Kim vocal 2)", "MDX-UVR Ins Model Full Band 292", "MDX-UVR Ins Model Full Band 403", "MDX-UVR Ins Model Full Band 450 (HQ_1)", "MDX-UVR Ins Model Full Band 498 (HQ_2)", "(de)Reverb HQ By FoxJoy"]
Demucs = 'off'#@param ["off","demucs_extra"]

#@markdown ---
#@markdown ### Parameters
denoise = False #@param {type:"boolean"}
normalise = True #@param {type:"boolean"}
#getting values from model_data.json related to ONNX var (model folder name)
amplitude_compensation = model_data[ONNX]["compensate"]
dim_f = model_data[ONNX]["mdx_dim_f_set"]
dim_t = model_data[ONNX]["mdx_dim_t_set"]
n_fft = model_data[ONNX]["mdx_n_fft_scale_set"]

mixing_algorithm = 'min_mag' #@param ["default","min_mag","max_mag"]
chunks = 55 #@param {type:"slider", min:1, max:55, step:1}
shifts = 10 #@param {type:"slider", min:0, max:10, step:0.1}

##validate values
track = track if 'http' in track else tracks_path+track
normalise = '--normalise' if normalise else ''
denoise = '--denoise' if denoise else ''

if ONNX == 'off':
    pass
else:
    ONNX = 'onnx/'+ONNX
if Demucs == 'off':
    pass
else:
    Demucs = 'model/'+Demucs+'.th'
#@markdown ---
#@markdown ### Stems
bass = False #@param {type:"boolean"}
drums = False #@param {type:"boolean"}
others = False #@param {type:"boolean"}
vocals = True #@param {type:"boolean"}
#@markdown ---
#@markdown ### Invert stems to mixture
invert_bass = False #@param {type:"boolean"}
invert_drums = False #@param {type:"boolean"}
invert_others = False #@param {type:"boolean"}
invert_vocals = True #@param {type:"boolean"}
invert_stems = []
stems = []
if bass:
    stems.append('b')
if drums:
    stems.append('d')
if others:
    stems.append('o')
if vocals:
    stems.append('v')

invert_stems = []
if invert_bass:
    invert_stems.append('b')
if invert_drums:
    invert_stems.append('d')
if invert_others:
    invert_stems.append('o')
if invert_vocals:
    invert_stems.append('v')

margin = 44100

###
# incompatibilities
###

console(f"python main.py --n_fft {n_fft} --dim_f {dim_f} --dim_t {dim_t} --margin {margin} -i \"{track}\" --mixing {mixing_algorithm} --onnx \"{ONNX}\" --model {Demucs} --shifts {round(shifts)} --stems {''.join(stems)} --invert {''.join(invert_stems)} --chunks {chunks} --compensate {amplitude_compensation} {normalise} {denoise}")

Model parameters (not necessary anymore)<br><br>
-*Fullband*: <br>self.n_fft = **6144** dim_f = **3072** dim_t = **8**<br><br>
-*kim vocals, ft other, inst 1-3 (415-464), 427*:<br> self.n_fft = **7680** dim_f = **3072** dim_t = **8**<br><br>
-*496, Karaoke, 9.X (NET-X)*<br>
self.n_fft = **6144** dim_f = **2048** dim_t = **8** (and **9** kuielab_a_vocals only)<br><br>
-Karaoke 2<br>
self.n_fft = **5120** dim_f = **2048** dim_t = **8**
<br><br>
You can use fullband config for e.g. kim ft other, so it will uncap the 17.7kHz cutoff, giving much less muddy output, but it will strongly amplify vocal residues (still, might be useful in some cases).
<br><br>
Volume compensation values (not necessary)<br><br>
-1.02/1.022099 is for Kim other ft (the 1st value recommended)<br>
-1.03 value is for "vocal" models e.g. 427 (and Kim vocal and 9.7 [NET 1]) <br>
-1.033/1.035437 is for HQ_2<br>
-1.05 *compensation* value is for "inst" models (e.g. 464 a.k.a. inst 3) <br>
-1.08 (old) for e.g. 418 model with normalization disabled and chunks 1 (less muddy result for min_mag, sometimes more instruments disappearing, also vs 464)<br>

The value above may differ also toward different tracks. Also, amplitude compensation doesn't change inverted stem.
- Don’t disable *invert_vocals* in Colab even if you only need one stem, otherwise the Colab will end up with error.<br>
- If you won't change any options above, these are the optimized settings for instrumentals with low amount of vocal residues and high quality (for less vocal residues but worse quality, you can also check inst3 and kim ft models).<br>
- Separating same track with different model will be replaced the old result (rename files manually if necessary)
- Inversion applied for second stem allowing getting vocals from instrumental models or vice versa - to get instrumentals from vocal models.
- Output file names for instrumental models will be swapped because the Colab originally was wrtitten for vocal models.<br> 
- In case of memory allocation error (especially with Demucs on), change chunks to 20 or 40 (the more chunks the lower vocal popups and sometimes vocal residues - fixed in UVR5 GUI in batch mode).<br> 
-In case of *system error* with input file name written, retry separation or ensure you wrote your file name correctly.<br>
-In case of *shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory* go to environment and terminate session and start over.<br>
- Downloading result files with Colab file manager can be (very) slow, and multiple file downloading may eventually download only one file - use GDrive instead. 
- Don't add shortcuts to tracks folder. The Colab will not output files properly.
<br>
<br>

**Settings explanation**<br>

*Mixing algorithm* - max_mag - is generally for vocals (gives the most residues in instrumentals), min_mag - for instrumentals (the most aggresive) though "min_mag solve some un-wanted vocal soundings, but instrumental [is] more muffled and less detailed". Check out also "default" as it's in between both - a.k.a. average (it's also required for Demucs enabled which works only for vocal models).<br>

*Chunks* - Set it to 55 or 40 (less aggressive) to alleviate some occasional instrument dissapearing.
Set 1 for the best clarity. It works for at least instrumental model (4:15 track, at least for Tesla T4 (shown at the top) generally better quality, but some instruments tend to disappear more using 1 than 10. For Demucs enabled and/or vocal model it can be set to 10 if your track is below 5:00 minutes. The more chunks, the faster separation up to ~40. For 4:15 track, 72 is max supported till memory allocation error shows up (disabled chunks returns error too). <br>

*Shifts* - can be set max to 10, but it only slightly increases SDR, while processing time is 1.7x longer for each shift and it gives similar result to shifts 5.

*Normalization* - "normalizes all input at first and then changes the wave peak back to original. This makes the separation process better, also less noise" (e.g. if you have to noisy hihats or big amplitude compensation - disable it).
<br>

*Demucs* enabled works correctly with mixing algorithm set to default and only with vocal models (Kim and 427). It's also the only option to get rid of noise of MDX models. Normalization enabled is necessary (but that cnfiguration has slightly more vocal residues than instrumental model). Decrease chunks to 40 if you have ONNXRuntimeError with Demucs on (it requires lower chunks).
<br><br>

*Recommended models*

For instrumentals:
- HQ_2 (best quality; normal vs min_mag less muddy, more vocal residues)

- kim ft other (if you have vocal residues with HQ models, but not always)

- inst 3/464 (if it's still a bit noisy)

- HQ_1 with 1.05 v. comp. (less muddy vs HQ_2 w/ v. 1.033, but more vocal residues, I like it)

For vocals:
- kim vocal (less instrumental residues in vocal stem)
<br>or alternatively
- 427 
- 406 (soon)

For lead vocals:
- Karaokee 2

For backing vocals:
- [HP_KAROKEE-MSB2-3BAND-3090](https://colab.research.google.com/drive/16Q44VBJiIrXOgTINztVDVeb0XKhLKHwl?usp=sharing)

<br>


Changelog
<br><br>
Kim's vocal model is "old" only in a sense that she already had a newer one at some point, but it was the best evaluated single vocal model by [SDR](https://mvsep.com/quality_checker/leaderboard.php?ensemble=0) available publically for 12.03.23 (now only surpassed by MDX23 on mvsep1.ru).

<sup>Added 292 full band beta instrumental model (no 14.7kHz cutoff). 292 epoch already [surpased](https://mvsep.com/quality_checker/leaderboard2.php?&sort=instrum&page=20) Demucs 4 ft model. 309 and 337 epochs are already better than inst 1 SDR-wise.

<sup>Newly added 403 full band beta instrumental model (no 14.7kHz cutoff) - should be better than the 293 in most if not all cases.<br></sup>
Final full band 450 (HQ_1) instrumental model addeded (it may leave more vocal residues than inst3 at times, but it's really worth trying out).<br>
Newer 498 epoch added (HQ_2) "better at removing long drawn out vocals"<br>
Kim other ft model added (the best SDR, even vs 464, but it has cutoff), read point 16.<br>
I've reverted old "Karokee" and "Karokee_AGGR" models, but these are old models (maybe they will do the trick, though), they're already in your MDX_Colab folder after mounting.<br>
Python 3.10 fix by Crusty Crab.<br>
Denoiser and model parameter picker by jarredou. <br>
Karaokee 2 model. Forgot adding it to the list.<br>
Torch 2.0 fix by jarredou. Colab installation speed-up.<br>
Fixed torch fix.

In case of persistent vocals.onnx error - manual installation of [model files](https://docs.google.com/document/d/17fjNvJzj8ZGSer7c7OFe_CNfUKbAxEh_OBv94ZdRG5c/edit#heading=h.aa2xhwp434) files.